_In this notebook, we create a dropdown menu to pre-configure the various experiments we might want to run in Netlogo. This notebook was originally built by Eric Shook (UMN) and modified by Colin D. Wren (UCCS)._


# Running Netlogo

We cannot unfortunately run Netlogo interactively with its graphical user interface in this Notebook, served via Binder. What we _can_ do is run Netlogo in 'headless' mode - that is to say, without the graphics! Why would we do that? It can be faster, but it also allows us to run experiments, sweeping through various parameter settings, and getting the output in a csv file that we can then import into other programs for analysis. Running Netlogo in headless mode requires some rather complicated command-line commands. Here, we're using the cell below to create some drop-down menus to allow us to execute our experiments by hitting the 'Run ABM Experiment' button.

We'll discuss how to set up your own experiments in a moment. For now, run the next code block:

In [1]:
import subprocess
import ipywidgets as widgets
from IPython.display import display

def runabm(model,experiment,tablename):
    subprocess.check_output("./run-abm-script "+model+" "+experiment+" "+tablename,shell=True)

wmodel = widgets.Dropdown(description='Model path and name:',
    options=['Virus.nlogo'])

wexp = widgets.Dropdown(description="BehaviorSpace experiment name:",
    options=['exp01', 'experiment1.xml'])

wtable = widgets.Text(description="Output table name:",
    value='test.csv')

def on_run_click(b):
    print("ABM experiment running ...")
    runabm(wmodel.value,wexp.value,wtable.value)
    print("ABM experiment complete.")
        
wrun = widgets.Button(description="Run ABM Experiment")
wrun.on_click(on_run_click)

display(wmodel,wexp,wtable,wrun)

Dropdown(description='Model path and name:', options=('Virus.nlogo',), value='Virus.nlogo')

Dropdown(description='BehaviorSpace experiment name:', options=('exp01', 'experiment1.xml'), value='exp01')

Text(value='test.csv', description='Output table name:')

Button(description='Run ABM Experiment', style=ButtonStyle())

ABM experiment running ...
ABM experiment complete.


...add hit that `Run ABM Experiment` button! Check the home page for the binder to see the results. If you'd like to see the Virus model running in Netlogo Web, right-click [this link](http://netlogoweb.org/launch#http://netlogoweb.org/assets/modelslib/Sample%20Models/Biology/Virus.nlogo) in a new browser window. 

## Adding other models & Experiments

Do you see what you would have to do to add more models, or more experiments, to the options? You'd just upload them to this folder and add their name:
```
wmodel = widgets.Dropdown(description='Model path and name:',
    options=['Virus.nlogo', 'your-model-name-here.nlogo'])
```

There are a number of models already loaded with netlogo - you can find them in this notebook by navigating: NetLogo6.0.4 -> app -> models -> Sample_Models. You'd add them like so:

```
    options=['Virus.nlogo', 'your-model-name-here.nlogo', '/home/jovyan/NetLogo 6.0.4/app/models/Sample_Models/Biology/Wolf_Sheep_Predation.nlogo'])
```

The experimental design is included in the .nlogo model code; we'll talk more about that in a moment. A model might have more than one experiments in it; each experiment has its own name and these can be added to the block above the same way the model names are.

Finally, you can write in the box the name of the output csv file for your experiment run.

## Designing another experiment

The experiment is marked up with xml at the end of the [Virus.nlogo](Virus.nlogo) file:

```
  <experiment name="exp01" repetitions="1" runMetricsEveryStep="true">
    <setup>setup</setup>
    <go>go</go>
    <exitCondition>(ticks / 52) = 10</exitCondition>
    <metric>count turtles with [ sick? ]</metric>
    <metric>count turtles with [ immune? ]</metric>
    <metric>count turtles with [ not sick? and not immune? ]</metric>
    <enumeratedValueSet variable="duration">
      <value value="20"/>
    </enumeratedValueSet>
    <enumeratedValueSet variable="turtle-shape">
      <value value="&quot;person&quot;"/>
    </enumeratedValueSet>
    <enumeratedValueSet variable="infectiousness">
      <value value="45"/>
      <value value="55"/>
      <value value="65"/>
      <value value="75"/>
    </enumeratedValueSet>
    <enumeratedValueSet variable="number-people">
      <value value="150"/>
    </enumeratedValueSet>
    <enumeratedValueSet variable="chance-recover">
      <value value="75"/>
    </enumeratedValueSet>
  </experiment>
```

To add another experiment to Virus, you copy and paste this code at the end of file just after the closing `</experiment>` tag, making sure that your new experiment has a new name:

```
 <experiment name="exp02" repetitions="1" runMetricsEveryStep="true">
```

But how do you know which values you can set, which parameters you can sweep through? If you look at the code for the model (or the [graphical representation](http://netlogoweb.org/launch#http://netlogoweb.org/assets/modelslib/Sample%20Models/Biology/Virus.nlogo)) you can see the names of all the variables that can be modified. In our original experiment, `infectiousness` had four separate settings - 45, 55, 65, 75. If instead we wanted to sweep through from a starting level to a finishing level in say increments of ten, we would need to specify the start, finish, and increment like this:
```
<steppedValueSet variable="infectiousness" first="45" step="10" last="75"/>
```

Notice also the `<metric>` tag. Inside this tag we put whatever Netlogo code we want that does the measuring. As archaeologists, one thing that we are often concerned with is how these complex systems evolve over time. How 'time' is defined in a model is another question altogether, but we often will want to count the 'ticks' of the NetLogo cycle. Open [Virus.nlogo](https://hub.mybinder.org/user/o-date-abm3-ir95qw2z/edit/Virus.nlogo) and add the following metric to the experiment:

```
<metric>ticks</metric>
```

And then re-run the model!


## Telling the experiment when to stop

Finally, notice also the `<exitCondition>` tag. We have to tell the model when to stop!

Try to define and add a second experiment to [Virus.nlogo](https://hub.mybinder.org/user/o-date-abm3-ir95qw2z/edit/Virus.nlogo). What combination of parameters would it be interesting to explore? Note that once you have more than one experiment, your tags will look like this:
```
<experiments>
  <experiment name="exp01" repetitions="1" runMetricsEveryStep="true">
    <setup>setup</setup>
    ...
    ...
  </experiment>
  <experiment name="exp02" repetitions="1" runMetricsEveryStep="true">
    <setup>setup</setup>
    ...
    ...
  </experiment>
</experiments>
```

## External experiment definition file

It is possible to put the experiment in an external file, and call that from our dropdowns above. In this binder you'll find the `experiment.xml` file which shows the extra bit of xml necessary to make the standalone file. **Warning** this currently is not working correctly in this binder, so define your experiment parameters *in* the nlogo file itself.